In [1]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [2]:
product_type = "ambient" #frozen chilled ambient 
num_time_periods = 7
storage_type="constrained" #standard constrained relaxed 
#Test in one article
test_article = "468a73f3"
M = 2000

In [3]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [4]:
# constants definitions
if(product_type=="frozen"):
    if(storage_type=="standard"):
        warehouse_volume = 30
    if(storage_type=="constrained"):
        warehouse_volume = 20
    if(storage_type=="relaxed"):
        warehouse_volume = 50
if(product_type=="chilled"):
    if(storage_type=="standard"):
        warehouse_volume = 250
    if(storage_type=="constrained"):
        warehouse_volume = 180
    if(storage_type=="relaxed"):
        warehouse_volume = 300
if(product_type=="ambient"):
    if(storage_type=="standard"):
        warehouse_volume = 700
    if(storage_type=="constrained"):
        warehouse_volume = 500
    if(storage_type=="relaxed"):
        warehouse_volume = 900
buffer_cost_per_m3 = 25
default_max_order = 10000
max_order_exceed_multiplier = 1.5

In [5]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [6]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [7]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [8]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [9]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

#### Heuristic

In [10]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [11]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = 0
        self.avg_cost = 0
        self.extra_orders = {}

        self.clearance_cost = 0
        self.buffer_cost = 0
        self.max_order_cost = 0
        self.fixed_cost = 0
        self.per_tu_cost = 0

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

In [12]:
cost_dict = {
    'fixed':0,
    'per_tu':0,
    'clearance':0,
    'buffer':0,
    'max_order_penalty':0,
    'total':0
}

In [13]:
n = len(demand.keys())
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    clearance_cost = 0
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            if(cell.order_date + shelf_life[item] < t):
                # item cannot be ordered on order_date because it would go bad by t
                if(t not in cell.extra_orders.keys()):
                    cell.extra_orders[t] = {}
                cell.extra_orders[t][item] = math.ceil(demand[t][item] / cu_per_tu[item])
                clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]
            else:   
                cell.orders[item] += demand[t][item]
                # cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    for item in cell.orders.keys():
        cell.orders[item] = math.ceil(cell.orders[item]/cu_per_tu[item])
        clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]

    cell.clearance_cost = clearance_cost

def fetchExtraVolume(cell, t):
    extra_volume = 0
    if(t in cell.extra_orders.keys()):
        for item in cell.extra_orders[t].keys():
            extra_volume += cell.extra_orders[t][item] * cu_per_tu[item] * volume_per_cu[item] 
    return extra_volume
            
def getCost(cell):
    getOrders(cell)
    cost = cell.clearance_cost
    volume = 0

    fixed_cost = 0
    per_tu_cost = 0
    buffer_cost = 0
    max_order_cost = 0

    for item in cell.orders.keys():
        # Fixed and per tu cost for items
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        fixed_cost += ordering_cost_fixed[item]
        per_tu_cost += ordering_cost_per_tu[item] * cell.orders[item]

        # Max order penalty added
        if(cell.orders[item] > maximum_order_quantity_tu[item]):
            cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])
            max_order_cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])

        # Add costs for extra orders
        for extra_order_date in cell.extra_orders.keys():
            for extra_item in cell.extra_orders[extra_order_date].keys():
                cost += (ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]) + ordering_cost_fixed[extra_item]
                fixed_cost += ordering_cost_fixed[extra_item]
                per_tu_cost += ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]
                
                # Max order penalty for extra orders
                if(cell.extra_orders[extra_order_date][extra_item] > maximum_order_quantity_tu[extra_item]):
                    cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    
                    max_order_cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    

        # Keep track of volume
        volume += cell.orders[item] * cu_per_tu[item] * volume_per_cu[item]
    
    # Checking for extra volume and adding buffer cost
    for t in range(cell.order_date,cell.period_end+1):
        if(volume + fetchExtraVolume(cell, t) > warehouse_volume):
            cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost_per_m3
            buffer_cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost_per_m3

            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            # Removing volume of demand sold that day
            volume -= demand_volume

            # Adding volume of extra items from that day
            volume += fetchExtraVolume(cell, t)
        else:
            break

    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1)

    cell.buffer_cost = buffer_cost
    cell.max_order_cost = max_order_cost
    cell.fixed_cost = fixed_cost
    cell.per_tu_cost = per_tu_cost

order_date = 0
end_reached = False
obj_val = 0
schedule = {}

while(True):
    # Skipping ordering on sundays
    if(order_date%7==6):
        order_date += 1
        continue

    # Find date *until* which you want to order
    min_cost = float('inf')
    for t in range(order_date,n):
        # Skip checking demand for sundays
        if(t%7==6):
            continue

        schedule[order_date] = {}
        getCost(cost_matrix[order_date][t])

        if(cost_matrix[order_date][t].avg_cost < min_cost):
            min_cost = cost_matrix[order_date][t].avg_cost
            # If looking at ordering until last day. Make the order
            if(t==n-1):
                schedule[order_date] = cost_matrix[order_date][t].orders
                end_reached = True
                obj_val += cost_matrix[order_date][t].cost

                cost_dict['buffer'] += cost_matrix[order_date][t].buffer_cost
                cost_dict['clearance'] += cost_matrix[order_date][t].clearance_cost
                cost_dict['fixed'] += cost_matrix[order_date][t].fixed_cost
                cost_dict['per_tu'] += cost_matrix[order_date][t].per_tu_cost
                cost_dict['max_order_penalty'] += cost_matrix[order_date][t].max_order_cost
                break
            
        # If cost increases, make order until previous day
        else:
            schedule[order_date] = cost_matrix[order_date][t-1].orders
            obj_val += cost_matrix[order_date][t-1].cost

            cost_dict['buffer'] += cost_matrix[order_date][t-1].buffer_cost
            cost_dict['clearance'] += cost_matrix[order_date][t-1].clearance_cost
            cost_dict['fixed'] += cost_matrix[order_date][t-1].fixed_cost
            cost_dict['per_tu'] += cost_matrix[order_date][t-1].per_tu_cost
            cost_dict['max_order_penalty'] += cost_matrix[order_date][t-1].max_order_cost

            # Place next order starting from today
            order_date = t
            break
    
    if(order_date >= n or end_reached):
        break

cost_dict['total'] = obj_val
cost_dict

{'fixed': 120212.7999999999,
 'per_tu': 740290.199999998,
 'clearance': 54168.00000000004,
 'buffer': 0,
 'max_order_penalty': 7423.799999999999,
 'total': 922094.7999999996}

In [14]:
result_dict = {}

for day, articles in schedule.items():
    for article, quantity in articles.items():
        if article in result_dict:
            result_dict[article][day] = quantity
        else:
            result_dict[article] = {day: quantity}
schedule = result_dict

In [15]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol

In [16]:
def neighbourhoodOperator1(schedule, obj_val):
    max_improvement = obj_val
    cost = obj_val
    modified_schedule = {}
    day_wise_orders = {}

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule

    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 3
    prev_max_improvement = 0

    while count < MAX_REPEAT and iterator < MAX_ITER:
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            cost_improvement = 0
            modified_orders = {}
            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i +1
                        
                    if order_days[day2] - order_days[day1] <= shelf_life[item]:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost_per_m3 * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        if delta < 0:
                            cost += delta
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            modified_orders[day1] = tot_orders
                            volumes[day1] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                modified_schedule[item] = modified_orders
            
            else: 
                if len(schedule[item]) == 1 :
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        print("Buffer penalty paid", buffer_penalty_tot)
        print("max order penalty paid", max_order_penalty_tot)
        print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        print(max_improvement)
        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement

    return schedule, obj_val, buffer_penalty_tot, max_order_penalty_tot, fixed_ordering_cost_tot

In [17]:
schedule, obj_val, buffer_cost_delta, max_order_penalty_delta, fixed_cost_delta  = neighbourhoodOperator1(schedule, obj_val)
cost_dict['buffer'] += buffer_cost_delta
cost_dict['max_order_penalty'] += max_order_penalty_delta
cost_dict['fixed'] -= fixed_cost_delta
cost_dict['total'] = obj_val
cost_dict

Buffer penalty paid 307.88448304929403
max order penalty paid 1.5
fixed cost reduced by 8897.9
913506.2844830491
Buffer penalty paid 307.88448304929403
max order penalty paid 1.5
fixed cost reduced by 8897.9
913506.2844830491
Buffer penalty paid 307.88448304929403
max order penalty paid 1.5
fixed cost reduced by 8897.9
913506.2844830491
Buffer penalty paid 307.88448304929403
max order penalty paid 1.5
fixed cost reduced by 8897.9
913506.2844830491


{'fixed': 111314.8999999999,
 'per_tu': 740290.199999998,
 'clearance': 54168.00000000004,
 'buffer': 307.88448304929403,
 'max_order_penalty': 7425.299999999999,
 'total': 913506.2844830491}

In [18]:
backup_schedule = schedule.copy()

In [19]:
schedule = backup_schedule.copy()

In [20]:
for item in schedule.keys():
    for day in schedule[item].keys():
        if(schedule[item][day] < 0):
            print("CRAP!")

In [21]:
def transposeScheduleToDayFirst(schedule):
    result_dict = {}
    for day, articles in schedule.items():
        for article, quantity in articles.items():
            if article in result_dict:
                result_dict[article][day] = quantity
            else:
                result_dict[article] = {day: quantity}
    return result_dict

def transposeScheduleToItemFirst(schedule):
    result_dict = {}
    for article, days in schedule.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][article] = quantity
            else:
                result_dict[day] = {article: quantity}
    return result_dict

In [22]:
shelf_life['59e744cc']

4

In [23]:
summ = 0
for day in demand.keys():
    if('59e744cc' in demand[day].keys()):
        summ += demand[day]['59e744cc']
        print(day, demand[day]['59e744cc'])
summ

0 25.0
1 20.0
2 23.0
3 36.0
4 35.0
5 14.0


153.0

In [24]:
# test_schedule = schedule.copy()
# # test_schedule = transposeScheduleToDayFirst({'bd82f3b0':test_schedule['bd82f3b0']})
# test_schedule = transposeScheduleToDayFirst({'59e744cc':test_schedule['59e744cc']})
# articles = ['59e744cc']
# test_schedule

In [25]:
new_clearance_cost = 0
ordering_cost_per_tu_saved = 0

# Add clearance cost
# If I ever have to throw away equal to or more than a TU's worth, then I can just reduce order
def removeExpired(inv,life,item):
    global ordering_cost_per_tu_saved
    global new_clearance_cost
    
    tu_saved = 0
    # if inventory has items from days which are now too old
    if(len(inv)>life):
        amount_cleared = inv[0]

        tu_saved = amount_cleared//cu_per_tu[item]
        # ordering cost reduced here
        ordering_cost_per_tu_saved += tu_saved * ordering_cost_per_tu[item]

        amount_inevitably_cleared = amount_cleared%cu_per_tu[item]
        # clearance cost recalculated here
        new_clearance_cost += amount_inevitably_cleared*clearing_cost_per_cu[item]
        
        # remove days which are now too old
        inv = inv[-life:]
    
    return inv, tu_saved

def addDay(inv,ordered):
    inv.append(ordered)
    return inv

# how much of the current demand can be satisfied from items in inventory
def satisfyFromInventory(inv, dem):
    for day in range(len(inv)):
        if(dem==0):
            break
        if(inv[day] <= dem):
            dem -= inv[day]
            inv[day] = 0
        else:
            inv[day] -= dem
            dem = 0
    return inv

def updateInventory(inv,item,t,ordered):
    life = shelf_life[item]
    dem = demand[t][item]
    # first the orders of the new day are added to the inventory
    # and then the total inventory is used to satisfy the demand
    # This would mean that the demand cannot go unsatisfied from
    # the items in the inventory
    inv = addDay(inv, ordered)
    inv, tu_saved = removeExpired(inv,life,item)
    inv = satisfyFromInventory(inv, dem)
    return inv, tu_saved

def calculateMaxOrderPenalty(schedule):
    max_order_penalty = 0
    for day in schedule.keys():
        for item in schedule[day].keys():
            if(schedule[day][item] > maximum_order_quantity_tu[item]):
                max_order_penalty += (schedule[day][item] - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
    return max_order_penalty

def neighbourhoodOperator2(schedule):
    # dictionary of format {'item':['q left on earliest possible','q left on earliest possible + 1',...,'q left today']}
    inventory = {}

    for t in time_indexes:
        for item in articles:

            if item not in inventory.keys():
                inventory[item] = []

            # if there was no order for an item in a particular day, then the order was 0
            if(t in schedule.keys() and item in schedule[t].keys()):
                ordered = schedule[t][item] * cu_per_tu[item]
            else:
                ordered = 0

            # update the inventory according to what has been ordered today for item
            inventory[item],tu_saved = updateInventory(inventory[item], item, t, ordered)
            
            # Removing excess TUs from the schedule. On day t, we remove the excess from day t-shelf life
            if(t-shelf_life[item] in schedule.keys() and item in schedule[t-shelf_life[item]].keys()):
                if(t>=shelf_life[item]):
                    schedule[t-shelf_life[item]][item] -= tu_saved

                
    print("Clearance cost: ", new_clearance_cost)
    print("Ordering cost per TU saved: ", ordering_cost_per_tu_saved)

    new_max_order_penalty = calculateMaxOrderPenalty(schedule)

    return schedule, new_clearance_cost, ordering_cost_per_tu_saved, new_max_order_penalty

In [26]:
schedule = transposeScheduleToDayFirst(schedule)
schedule, new_clearance_cost, per_tu_delta, new_max_order_penalty = neighbourhoodOperator2(schedule)
cost_dict['clearance'] = new_clearance_cost
cost_dict['max_order_penalty'] = new_max_order_penalty
cost_dict['per_tu'] -= per_tu_delta
cost_dict['total'] = cost_dict['fixed'] + cost_dict['per_tu'] + cost_dict['buffer'] + cost_dict['clearance'] + cost_dict['max_order_penalty']

Clearance cost:  585.4999999999999
Ordering cost per TU saved:  18683.500000000004


In [27]:
cost_dict

{'fixed': 111314.8999999999,
 'per_tu': 721606.699999998,
 'clearance': 585.4999999999999,
 'buffer': 307.88448304929403,
 'max_order_penalty': 7423.799999999999,
 'total': 841238.7844830472}